# Working with Raw Gaze Samples

Once gaze data have been loaded, they are available as time-ordered raw samples in `gaze.samples`. 

The table below shows a basic example of raw gaze samples after import into `pymovemnts`. Each row corresponds to one time-ordered gaze sample and is stored in the ``samples`` attribute of the {py:class}`~pymovements.Gaze` object. Timestamps are listed in the ``time`` column, and horizontal and vertical gaze positions in pixel coordinates can be found in the `pixel` column. Depending on the loader and input format, additional channels such as binocular coordinates or quality measures may also be present.

In [ ]:
import matplotlib.pyplot as plt

import pymovements as pm
from pymovements.gaze.experiment import Experiment

experiment = Experiment(
    screen_width_px=1680,
    screen_height_px=1050,
    screen_width_cm=47.5,
    screen_height_cm=30,
    distance_cm=65,
    origin='upper left',
    sampling_rate=1000.0,
)

gaze = pm.gaze.from_csv(
    "../examples/gaze-toy-example.csv",
    experiment=experiment,
    time_column="timestamp",
    pixel_columns=['x', 'y']
)

gaze.samples.head(5)

## Inspecting Raw Samples with Plots

Visual inspection is an essential first step when working with newly loaded gaze data. Time-series plots help reveal signal loss, noise, blinks, sampling irregularities, or calibration problems before any preprocessing is applied. Using the {py:func}`~pymovements.plotting.traceplot` function, we can visualize raw gaze samples from a {py:class}`~pymovements.Gaze` object. The plot shows the continuous trajectory of gaze positions across the stimulus, allowing inspection of spatial gaze behavior over time.

In [ ]:
pm.plotting.traceplot(gaze)

plt.show()

We can examine how each recorded signal changes over time by using the {py:func}`~pymovements.plotting.tsplot` function. It produces a time-series plot with one line per selected channel (e.g., horizontal and vertical gaze position). The x-axis represents time, as defined by the gaze sample timestamps. In this example, we plot the `x` and `y` channels. 

In [ ]:
gaze_unnested = gaze.clone()
gaze_unnested.unnest('pixel')

pm.plotting.tsplot(
    gaze_unnested,
    channels=['pixel_x', 'pixel_y'],
    share_y=False,
    line_color="darkblue",
    zero_centered_yaxis=False)

plt.show()

## Transforming Raw Samples

Raw pixel coordinates are tied to a specific screen setup and viewing distance. For meaningful interpretation and cross-experiment comparison, gaze samples are often transformed into alternative representations. These transformations operate directly on the raw samples and rely on the experimental metadata defined earlier.

### `pix2deg()`: From Pixels to Degrees of Visual Angle

Eye trackers typically record gaze positions in screen pixels. While useful for display-based inspection, pixel units depend on screen size and viewing distance and are therefore not comparable across setups. The {py:func}`~pymovements.gaze.transforms.pix2deg` function converts pixel coordinates into degrees of visual angle (dva) using the experiment's screen geometry and viewing distance.

Requirements:
- A pixel-based gaze column must be available (by default named "pixel")
- An {py:class}`~pymovements.Experiment` must be attached to the gaze data, because screen size and distance are needed for the conversion

In [ ]:
gaze.pix2deg()
gaze

Unlike pixels, degrees of visual angle reflect the actual angular displacement of the eye relative to the observer's viewpoint and are therefore comparable across different screen setups and viewing distances. In the plot below, the overall shape of the signal remains the same as in pixel space, since only the unit of measurement has changed. However, the scale of the y-axes differs, reflecting the conversion from screen-dependent coordinates to angular units.

In [ ]:
gaze_unnested = gaze.clone()
gaze_unnested.unnest('position')

pm.plotting.tsplot(
    gaze_unnested,
    channels=['position_x', 'position_y'],
    share_y=False,
    line_color="darkblue",
    n_rows=2, n_cols=1,
    zero_centered_yaxis=False)

plt.show()

### `pos2vel()`: From Position to Velocity

Many eye-movement measures are derived not from position directly but from its temporal
derivatives. Velocity is computed from changes in gaze position over time and is central to event detection algorithms for saccades and fixations. In pymovements, velocity is computed explicitly from position data with the {py:func}`~pymovements.gaze.transforms.pos2vel` function, using the sampling rate stored in the eye tracker definition.

In [ ]:
gaze.pos2vel()
gaze

The following plot illustrates velocity, i.e. how quickly the eye moves at each time point. Periods of relative stability (low velocity) typically correspond to fixations, whereas sharp peaks in the signal indicate rapid eye movements such as saccades.

In [ ]:
gaze_unnested = gaze.clone()
gaze_unnested.unnest('velocity')

pm.plotting.tsplot(
    gaze_unnested,
    channels=['velocity_x', 'velocity_y'],
    share_y=False,
    line_color="darkblue",
    n_rows=2, n_cols=1,
    zero_centered_yaxis=False)

plt.show()

For more information on these preprocessing steps, please see the {doc}`Preprocessing Raw Gaze Data <../tutorials/preprocessing-raw-data>`